In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('SGD_TESTING.csv')
print(df)
dic = {}

for fr, to, time, type, user_id in zip(df["from"], df["to"], df["timestamp"], df["type"], df["user_id"]):
    if user_id in dic:
        dic[user_id].append([fr, to, type])
    else:
        dic[user_id] = [[fr, to, type]]


hashes = []
for x in range(1,len(dic)):
    dic[x] = np.array(dic[x]).flatten()
    hashes.append(dic[x])


print(len(dic))

      from    to                timestamp type  user_id
0      NaN    S0  2024-05-26 14:07:50.000  Req        1
1       S0    S1  2024-05-26 14:07:50.000  Req        1
2       S1  S1_2  2024-05-26 14:07:50.000  Req        1
3     S1_2    S1  2024-05-26 14:07:50.003  Res        1
4       S1    S0  2024-05-26 14:07:50.003  Res        1
...    ...   ...                      ...  ...      ...
9381    S0    S4  2024-06-15 09:49:30.024  Req      361
9382    S4  S4_2  2024-06-15 09:49:30.024  Req      361
9383  S4_2    S4  2024-06-15 09:49:30.034  Res      361
9384    S4    S0  2024-06-15 09:49:30.034  Res      361
9385    S0   NaN  2024-06-15 09:49:30.034  Res      361

[9386 rows x 5 columns]
361


In [3]:
from random import shuffle
import math
def jaccard_similarity(set1, set2):
    # intersection of two sets
    intersection = len(np.intersect1d(set1, set2))
    # Unions of two sets
    union = len(np.union1d(set1, set2))
    if union == 0:
        return 0
    else:
        return intersection / union

def create_hash_func(size: int, vocab):
    # function for creating the hash vector/function
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int, vocab : int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size, vocab))
    return hashes

def create_hash(vector: list, vocab, minhash_func):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature


def shingle(text, k):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)



In [5]:
#shingle
print("".join(dic[1]))
print("".join(dic[6]))

arr1 = shingle("".join(dic[1]), 3)
arr2 = shingle("".join(dic[6]), 3)

print(arr1)
print(arr2)
#vocab for 1hot
vocab = list(arr1.union(arr2))
#print(vocab)

#1hot-encoding
x = [1 if item in arr1 else 0 for item in vocab]
y = [1 if item in arr2 else 0 for item in vocab]

print(x,y)


minhash_func = build_minhash_func(len(vocab), 100, vocab)

arr1_sig = create_hash(x, vocab, minhash_func)
arr2_sig = create_hash(y, vocab, minhash_func)

print(arr1_sig)
print(arr2_sig)

print(jaccard_similarity(arr1_sig, arr2_sig))
print(jaccard_similarity(dic[1], dic[6]))



nanS0ReqS0S1ReqS1S1_2ReqS1_2S1ResS1S0ResS0S2ReqS2S2_3ReqS2_3S2ResS2S0ResS0S3ReqS3S3_1ReqS3_1S3ResS3S3_2ReqS3_2S3ResS3S3_3ReqS3_3S3ResS3S0ResS0S4ReqS4S4_2ReqS4_2S4ResS4S0ResS0nanRes
nanS0ReqS0S1ReqS1S1_3ReqS1_3S1ResS1S0ResS0S2ReqS2S2_2ReqS2_2S2ResS2S2_3ReqS2_3S2ResS2S0ResS0S3ReqS3S3_1ReqS3_1S3ResS3S3_2ReqS3_2S3ResS3S3_3ReqS3_3S3ResS3S0ResS0S4ReqS4S4_1ReqS4_1S4ResS4S0ResS0nanRes
{'Req', 'S0n', '2S0', '1S0', 'S1S', '1S1', '3S2', '3_2', '2S4', '4S0', '0na', '0S2', 'qS0', '_1R', '2S1', '0S4', 'S3S', 'esS', 'sS0', 'qS2', 'S4R', 'S2R', '3_3', '4Re', 'nRe', '3_1', 'anS', 'S3_', '1S3', '2S3', 'sS3', 'qS1', 'S0S', 'sS1', 'S2_', '0S3', 'S4S', 'S2S', 'eqS', '3Re', '2_3', 'S4_', 'S1_', 'anR', '4S4', '_3S', '_3R', 'S3R', 'sS4', '2Re', 'nS0', '1_2', '2S2', '4_2', '1Re', '_1S', '0Re', 'qS4', 'Res', 'nan', '_2R', '0S1', '3S3', '3S0', 'S1R', '_2S', 'qS3', 'S0R', 'sS2'}
{'Req', 'S0n', '2S0', '1S0', 'S1S', '1S1', '3S2', '3_2', '4S0', '0na', '0S2', 'qS0', '1_3', '_1R', '0S4', 'S3S', 'esS', 'sS0', 'qS2', 'S

In [6]:
signatures = []
for x in range(1,len(dic)):
    for y in range(x+1, len(dic)-1):
        arr1 = shingle("".join(dic[x]), 3)
        arr2 = shingle("".join(dic[y]), 3)
        vocab = list(arr1.union(arr2))
        arr1_hot = [1 if item in arr1 else 0 for item in vocab]
        arr2_hot = [1 if item in arr2 else 0 for item in vocab]
        minhash_func = build_minhash_func(len(vocab), 20, vocab)
        arr1_sig = create_hash(arr1_hot, vocab, minhash_func)
        arr2_sig = create_hash(arr2_hot, vocab, minhash_func)
        jacc = jaccard_similarity(arr1_sig, arr2_sig)
        print(x,y,jacc)
        signatures.append([x,y, jacc])

df = pd.DataFrame(signatures)
df.head()
        

        

1 2 0.46153846153846156
1 3 0.7272727272727273
1 4 0.7894736842105263
1 5 0.7619047619047619
1 6 0.8095238095238095
1 7 0.9047619047619048
1 8 0.4583333333333333
1 9 0.7894736842105263
1 10 0.8095238095238095
1 11 0.8
1 12 0.9444444444444444
1 13 0.47619047619047616
1 14 0.8947368421052632
1 15 0.6363636363636364
1 16 0.625
1 17 0.6666666666666666
1 18 0.8
1 19 0.7
1 20 0.6818181818181818
1 21 0.48
1 22 0.6956521739130435
1 23 0.5
1 24 0.6190476190476191
1 25 0.6521739130434783
1 26 0.6666666666666666
1 27 0.5
1 28 0.5
1 29 0.52
1 30 0.7272727272727273
1 31 0.65
1 32 1.0
1 33 1.0
1 34 0.5652173913043478
1 35 1.0
1 36 0.5652173913043478
1 37 0.7619047619047619
1 38 0.5416666666666666
1 39 1.0
1 40 0.6818181818181818
1 41 0.7272727272727273
1 42 0.7142857142857143
1 43 0.6086956521739131
1 44 0.8
1 45 0.6818181818181818
1 46 0.7391304347826086
1 47 0.37037037037037035
1 48 0.6521739130434783
1 49 0.8421052631578947
1 50 0.6521739130434783
1 51 0.6470588235294118
1 52 0.6666666666666666
1

,0,1,2
0,1,2,0.461538
1,1,3,0.727273
2,1,4,0.789474
3,1,5,0.761905
4,1,6,0.809524


In [20]:
from datasketch import MinHash, MinHashLSH

lsh = MinHashLSH(threshold=0.8, num_perm=128)

minhashes = {}

for user_id in dic:
    # Convert all elements to strings
    flattened_list = ["".join(map(str, sublist)) for sublist in dic[user_id]]
    shingles = shingle("".join(flattened_list), 3)
    m = MinHash(num_perm=128)
    for shingle_item in shingles:
        m.update(shingle_item.encode('utf8'))
    minhashes[user_id] = m
    lsh.insert(f"user_{user_id}", m)

def find_neighbors(user_id):
    if user_id in minhashes:
        result = lsh.query(minhashes[user_id])
        return result
    else:
        return []

user_id_check = 2
neighbors = find_neighbors(user_id_check)
print(f"LSH neighbors for user {user_id_check}:", neighbors)


LSH neighbors for user 2: ['user_174', 'user_334', 'user_25', 'user_261', 'user_93', 'user_210', 'user_248', 'user_136', 'user_308', 'user_200', 'user_239', 'user_146', 'user_205', 'user_36', 'user_38', 'user_20', 'user_10', 'user_56', 'user_348', 'user_27', 'user_272', 'user_321', 'user_65', 'user_225', 'user_3', 'user_309', 'user_247', 'user_193', 'user_17', 'user_299', 'user_203', 'user_105', 'user_94', 'user_336', 'user_68', 'user_186', 'user_288', 'user_305', 'user_140', 'user_297', 'user_77', 'user_333', 'user_219', 'user_290', 'user_357', 'user_282', 'user_2', 'user_224', 'user_214', 'user_54', 'user_139', 'user_320', 'user_331', 'user_183', 'user_8', 'user_230', 'user_42', 'user_202', 'user_112', 'user_240', 'user_19', 'user_109', 'user_48', 'user_128', 'user_115', 'user_316', 'user_168', 'user_338', 'user_155', 'user_39', 'user_310', 'user_319', 'user_52', 'user_37', 'user_281', 'user_162', 'user_207', 'user_277', 'user_79', 'user_76', 'user_294', 'user_149', 'user_178', 'user

In [7]:
import numpy as np
from sklearn.utils import murmurhash3_32
import pandas as pd
from collections import defaultdict

def lsh_hash_signature(signature, num_bands, rows_per_band):
    """
    Hashes a MinHash signature into buckets using Locality Sensitive Hashing (LSH).

    Parameters:
    signature (list): The MinHash signature.
    num_bands (int): Number of bands.
    rows_per_band (int): Number of rows per band.

    Returns:
    list: A list of hash values, one for each band.
    """
    hash_bands = []
    for i in range(num_bands):
        start_index = i * rows_per_band
        end_index = start_index + rows_per_band
        band = tuple(signature[start_index:end_index])
        hash_bands.append(murmurhash3_32(band))
    return hash_bands

def bucket_signatures(signatures, num_bands, rows_per_band):
    """
    Buckets MinHash signatures into LSH buckets.

    Parameters:
    signatures (list): List of MinHash signatures.
    num_bands (int): Number of bands.
    rows_per_band (int): Number of rows per band.

    Returns:
    dict: A dictionary where keys are bucket identifiers and values are lists of indices of signatures in that bucket.
    """
    buckets = defaultdict(list)
    for index, signature in enumerate(signatures):
        hash_bands = lsh_hash_signature(signature, num_bands, rows_per_band)
        for band_hash in hash_bands:
            buckets[band_hash].append(index)
    return buckets

def compare_buckets(buckets, signatures):
    """
    Compare items within the same LSH bucket to identify similar pairs.

    Parameters:
    buckets (dict): LSH buckets.
    signatures (list): List of MinHash signatures.

    Returns:
    list: List of tuples representing pairs of similar items and their Jaccard similarity.
    """
    similar_pairs = []
    for bucket, indices in buckets.items():
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                index1, index2 = indices[i], indices[j]
                jacc = jaccard_similarity(signatures[index1], signatures[index2])
                similar_pairs.append((index1, index2, jacc))
    return similar_pairs




# Assuming you already have a function to create MinHash signatures, e.g., create_minhash_signature
# and a function to calculate Jaccard similarity, e.g., jaccard_similarity

# Example: MinHash signatures for your items
#signatures = [create_minhash_signature(shingle("".join(dic[i]), 3), 100) for i in range(1, len(dic))]#

# Parameters for LSH
num_bands = 20
rows_per_band = 5

# Bucket signatures using LSH
buckets = bucket_signatures(signatures, num_bands, rows_per_band)

# Compare items within each bucket
similar_pairs = compare_buckets(buckets, signatures)

# Create a DataFrame to store the results
df = pd.DataFrame(similar_pairs, columns=["Index1", "Index2", "JaccardSimilarity"])
print(df.head())


TypeError: key (1, 2, 0.46153846153846156) with type <class 'tuple'> is not supported. Explicit conversion to bytes is required